In [1]:
import os
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.datasets.sleep_physionet.age import fetch_data

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
import antropy as ant
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pandas as pd
from pycaret.classification import *
from scipy.stats import skew, kurtosis


In [15]:
def process_edf(subject):
    EEG_path = "C:/Users/utkar/Desktop/Research_Paper/EEG Analysis/Sleep_Stage_Detection/sleep-edf-database-expanded-1.0.0/sleep-edf-database-expanded-1.0.0/sleep-telemetry/"
    raw=None
    annot_train=None
    files = os.listdir(EEG_path+subject)
    for file in os.listdir(EEG_path+subject):
        if (file.split('-')[1].split('.')[0] == 'PSG'):
            raw = mne.io.read_raw_edf(EEG_path+subject+'/'+file, stim_channel='Marker', preload=True, verbose=False)
        else:
            annot_train = mne.read_annotations(EEG_path+subject+'/'+file)
            raw.set_annotations(annot_train, emit_warning=False)

    raw.set_channel_types({'EOG horizontal': 'eog', 'EEG Fpz-Cz': 'eeg', 'EEG Pz-Oz': 'eeg', 'EMG submental': 'misc'})
    raw.drop_channels(['Marker'])

    raw.filter(0.3, 40, n_jobs=4)  # Filter the data with a 0.3-40 Hz bandpass
    annot_events = {
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3': 3,
    'Sleep stage 4': 3,
    'Sleep stage R': 4,
    }


    # keep last 30-min wake events before sleep and first 30-min wake events after
    # sleep and redefine annotations on raw data
    annot_train.crop(annot_train[0]['onset'] - 30 * 60,
                    annot_train[-2]['onset'] + 30 * 60)
    raw_train = raw.copy()
    raw_train.set_annotations(annot_train, emit_warning=False)
    events_train, _ = mne.events_from_annotations(raw_train, event_id = annot_events, chunk_duration=30.)

    # create a new event_id that unifies stages 3 and 4
    event_id = {'Sleep stage W': 0,
                'Sleep stage 1': 1,
                'Sleep stage 2': 2,
                'Sleep stage 3/4': 3,
                'Sleep stage R': 4}

    # plot events
    # fig = mne.viz.plot_events(events_train, event_id=event_id,
    #                         sfreq=raw_train.info['sfreq'],
    #                         first_samp=events_train[0, 0])

    # keep the color-code for further plotting
    stage_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    # Create epochs from the event data with 30sec time window
    tmax = 30. - 1. / raw_train.info['sfreq']  # tmax in included

    epochs_train = mne.Epochs(raw=raw_train, events=events_train,
                            event_id=event_id, tmin=0., tmax=tmax, baseline=None, preload=True)
    psds, freqs = mne.time_frequency.psd_welch(epochs_train, fmin=0.5, fmax=40., verbose=False)



    n_epochs = len(epochs_train)
    n_channels = len(epochs_train.ch_names)
    epoch_length = tmax

    svd_entropies = np.zeros((n_epochs, n_channels))
    hfd = np.zeros((n_epochs, n_channels))
    perm_ent = np.zeros((n_epochs, n_channels))
    det_fluc = np.zeros((n_epochs, n_channels))


    for i, epoch in enumerate(epochs_train.get_data()):
        for j, channel_data in enumerate(epoch):
            svd_entropies[i, j] = ant.svd_entropy(channel_data, normalize=True)
            hfd[i, j] = ant.higuchi_fd(channel_data)
            perm_ent[i, j] = ant.perm_entropy(channel_data, order=3, normalize=True)
            det_fluc[i, j] = ant.detrended_fluctuation(channel_data)

    X_svd = svd_entropies
    X_hfd = hfd
    X_perm = perm_ent
    X_det = det_fluc
    Y = epochs_train.events[:, -1]
    X_PSD = psds

    data = epochs_train.get_data()

    mean = np.mean(data, axis=2)
    std_dev = np.std(data, axis=2)
    skewness = skew(data, axis=2)
    kurt = kurtosis(data, axis=2)

    X_time_domain = np.concatenate((mean, std_dev, skewness, kurt), axis=1)

    return X_svd, X_hfd, X_perm, X_det, X_PSD, X_time_domain, Y

In [16]:
X_all_svd, X_all_hfd, X_all_perm, X_all_det, X_all_PSD, X_all_time, Y_all = [], [], [], [], [], [], []
EEG_path = "C:/Users/utkar/Desktop/Research_Paper/EEG Analysis/Sleep_Stage_Detection/sleep-edf-database-expanded-1.0.0/sleep-edf-database-expanded-1.0.0/sleep-telemetry/"

for subject in tqdm(os.listdir(EEG_path)):
    tqdm.write(subject, end='\r')
    X1, X2, X3, X4, X5, X6, Y = process_edf(subject)
    X_all_svd.append(X1)
    X_all_hfd.append(X2)
    X_all_perm.append(X3)
    X_all_det.append(X4)
    X_all_PSD.append(X5)
    X_all_time.append(X6)
    Y_all.append(Y)

  0%|          | 0/43 [00:00<?, ?it/s]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    2.2s finished


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1042 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1042 events and 3000 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.0s finished


...

Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
931 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 931 events and 3000 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s finished


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
943 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 943 events and 3000 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s finished


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
923 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 923 events and 3000 original time points ...
0 bad epochs dropped


In [17]:
X_all_time_arr = np.vstack(X_all_time)
X_all_svd_arr = np.vstack(X_all_svd)
X_all_hfd_arr = np.vstack(X_all_hfd)

X_all_perm_arr = np.vstack(X_all_perm)
X_all_det_arr = np.vstack(X_all_det)
X_all_PSD_arr = np.vstack(X_all_PSD)
X_all_PSD_arr = X_all_PSD_arr.reshape(41490, -1)
Y_all_arr = np.concatenate(Y_all)

In [18]:
print(X_all_time_arr.shape, X_all_svd_arr.shape, X_all_hfd_arr.shape, X_all_perm_arr.shape, X_all_det_arr.shape, X_all_PSD_arr.shape, Y_all_arr.shape)

(41490, 16) (41490, 4) (41490, 4) (41490, 4) (41490, 4) (41490, 202) (41490,)


In [29]:
df_X_time = pd.DataFrame(X_all_time_arr)
df_X_svd = pd.DataFrame(X_all_svd_arr)
df_X_hfd = pd.DataFrame(X_all_hfd_arr)
df_X_perm = pd.DataFrame(X_all_perm_arr)
df_X_det = pd.DataFrame(X_all_det_arr)

df_X_PSD = pd.DataFrame(X_all_PSD_arr)
df_Y = pd.DataFrame(Y_all_arr)


df_X_svd.columns = ['EEG Fpz-Cz_svd', 'EEG Pz-Oz_svd', 'EOG horizontal_svd', 'EMG submental_svd']
df_X_hfd.columns = ['EEG Fpz-Cz_hfd', 'EEG Pz-Oz_hfd', 'EOG horizontal_hfd', 'EMG submental_hfd']
df_X_perm.columns = ['EEG Fpz-Cz_perm', 'EEG Pz-Oz_perm', 'EOG horizontal_perm', 'EMG submental_perm']
df_X_det.columns = ['EEG Fpz-Cz_det', 'EEG Pz-Oz_det', 'EOG horizontal_det', 'EMG submental_det']
df_X_time.columns = ['EEG Fpz-Cz_mean', 'EEG Pz-Oz_mean', 'EOG horizontal_mean', 'EMG submental_mean',\
                        'EEG Fpz-Cz_std', 'EEG Pz-Oz_std', 'EOG horizontal_std', 'EMG submental_std',\
                        'EEG Fpz-Cz_skew', 'EEG Pz-Oz_skew', 'EOG horizontal_skew', 'EMG submental_skew',\
                        'EEG Fpz-Cz_kurt', 'EEG Pz-Oz_kurt', 'EOG horizontal_kurt', 'EMG submental_kurt']
df_Y.columns = ['Sleep Stage']

In [30]:
df_input = pd.concat([df_X_svd, df_X_hfd, df_X_perm, df_X_det, df_X_PSD, df_X_time, df_Y ], axis=1)

In [31]:
df_input

,EEG Fpz-Cz_svd,EEG Pz-Oz_svd,EOG horizontal_svd,EMG submental_svd,EEG Fpz-Cz_hfd,EEG Pz-Oz_hfd,EOG horizontal_hfd,EMG submental_hfd,EEG Fpz-Cz_perm,EEG Pz-Oz_perm,EOG horizontal_perm,EMG submental_perm,EEG Fpz-Cz_det,EEG Pz-Oz_det,EOG horizontal_det,EMG submental_det,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,EEG Fpz-Cz_mean,EEG Pz-Oz_mean,EOG horizontal_mean,EMG submental_mean,EEG Fpz-Cz_std,EEG Pz-Oz_std,EOG horizontal_std,EMG submental_std,EEG Fpz-Cz_skew,EEG Pz-Oz_skew,EOG horizontal_skew,EMG submental_skew,EEG Fpz-Cz_kurt,EEG Pz-Oz_kurt,EOG horizontal_kurt,EMG submental_kurt,Sleep Stage
0,0.987039,0.981167,0.793655,0.999226,1.729953,1.892506,1.756279,1.954969,0.946917,0.955711,0.953098,0.963475,0.624881,0.887048,1.148590,0.444626,5.553050e-10,3.290357e-10,3.626885e-10,3.081483e-10,2.405295e-10,1.998294e-10,1.807437e-10,1.387022e-10,1.094322e-10,8.776934e-11,6.622015e-11,3.847705e-11,5.690233e-11,3.107349e-11,7.178277e-11,6.745937e-11,8.797761e-11,3.731378e-11,5.700621e-11,4.957495e-11,7.524095e-11,5.605747e-11,7.722828e-11,7.629879e-11,9.252240e-11,8.915048e-11,1.175848e-10,1.165245e-10,1.345788e-10,1.082207e-10,1.323859e-10,1.024284e-10,8.602643e-11,7.831696e-11,7.585280e-11,5.297959e-11,3.894393e-11,3.333449e-11,3.024244e-11,2.219447e-11,1.729353e-11,1.246984e-11,1.359954e-11,1.194308e-11,8.439562e-12,9.224394e-12,3.861112e-12,4.160874e-12,5.565756e-12,5.083729e-12,1.792866e-11,2.108277e-11,4.310760e-11,6.100985e-11,9.031157e-11,1.217580e-10,1.505040e-10,1.832330e-10,2.285239e-10,2.466557e-10,2.819318e-10,3.260881e-10,3.496395e-10,3.257449e-10,3.439173e-10,3.406892e-10,3.407103e-10,3.201636e-10,3.147224e-10,3.220985e-10,3.344932e-10,3.162012e-10,3.270710e-10,3.475254e-10,3.673424e-10,3.787899e-10,4.018216e-10,3.989876e-10,4.092605e-10,3.977370e-10,3.883350e-10,3.529095e-10,3.480039e-10,3.100330e-10,2.543965e-10,2.090970e-10,1.805485e-10,1.416308e-10,9.712567e-11,6.028872e-11,4.408566e-11,3.024627e-11,2.748551e-11,3.184614e-11,3.952600e-11,4.805155e-11,6.737631e-11,8.070086e-11,8.565522e-11,9.239853e-11,9.727427e-11,9.000780e-10,1.606427e-10,5.272756e-11,4.682655e-11,6.205924e-11,4.595133e-11,4.349384e-11,5.405309e-11,5.906766e-11,5.905030e-11,6.737108e-11,4.071948e-11,4.824927e-11,2.717645e-11,2.140516e-11,7.912514e-12,6.129157e-12,1.065658e-11,2.483474e-11,3.512541e-11,3.847786e-11,8.276491e-11,7.220284e-11,9.583952e-11,7.948307e-11,7.869551e-11,6.128152e-11,4.804111e-11,3.209743e-11,1.649746e-11,5.152072e-12,2.546775e-12,3.851909e-12,1.146667e-11,2.397788e-11,2.688461e-11,2.738034e-11,4.283229e-11,5.804657e-11,4.995176e-11,7.547974e-11,7.769232e-11,7.629224e-11,9.467743e-11,9.301742e-11,7.183464e-11,8.254202e-11,9.193125e-11,1.073197e-10,9.626151e-11,1.027942e-10,1.124286e-10,1.413720e-10,1.679854e-10,2.173989e-10,2.596037e-10,2.484419e-10,2.795427e-10,2.996474e-10,2.692400e-10,2.092549e-10,1.898273e-10,1.471153e-10,9.564443e-11,7.218853e-11,5.304065e-11,6.089144e-11,6.327405e-11,7.808140e-11,1.063323e-10,1.002388e-10,1.133735e-10,1.264167e-10,8.702129e-11,7.119187e-11,6.719815e-11,4.233593e-11,1.062860e-11,8.300893e-12,4.151642e-12,3.510404e-12,4.707987e-12,1.398299e-11,2.300863e-11,3.790104e-11,3.750684e-11,5.123753e-11,7.609510e-11,7.186003e-11,7.709495e-11,9.520194e-11,1.058957e-10,7.870332e-11,7.839036e-11,7.018454e-11,3.721077e-11,4.151750e-11,3.966558e-11,3.782345e-11

In [32]:
df_input = df_input.fillna(0)

In [33]:
clf = setup(
    data = df_input,
    target = 'Sleep Stage',
    normalize = True,
    fix_imbalance=True
    )

,Description,Value
0,session_id,7075
1,Target,Sleep Stage
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(41490, 235)"
5,Missing Values,False
6,Numeric Features,234
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [34]:
compare_models(fold=15)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.8744,0.9793,0.8547,0.8777,0.8755,0.8234,0.8238,177.7513
lightgbm,Light Gradient Boosting Machine,0.8714,0.9791,0.8561,0.8764,0.8730,0.8198,0.8204,12.3633
rf,Random Forest Classifier,0.8614,0.9743,0.8396,0.8661,0.8630,0.8051,0.8055,15.4633
et,Extra Trees Classifier,0.8600,0.9732,0.8329,0.8612,0.8601,0.8021,0.8023,9.6213
gbc,Gradient Boosting Classifier,0.8417,0.9717,0.8405,0.8553,0.8454,0.7810,0.7833,223.0213
svm,SVM - Linear Kernel,0.7922,0.0000,0.7954,0.8166,0.7979,0.7159,0.7207,5.8793
knn,K Neighbors Classifier,0.7860,0.9295,0.7940,0.8227,0.7940,0.7109,0.7187,18.8880
lda,Linear Discriminant Analysis,0.7688,0.9387,0.7622,0.7957,0.7771,0.6826,0.6863,6.2940
dt,Decision Tree Classifier,0.7642,0.8410,0.7402,0.7770,0.7688,0.6722,0.6736,8.5440
ridge,Ridge Classifier,0.7401,0.0000,0.7493,0.7739,0.7459,0.6492,0.6572,5.0680


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=7075, reg_alpha=0, ...)

## 4 Stages - Wake, Light, Deep and REM

In [35]:
def process_edf(subject):
    EEG_path = "C:/Users/utkar/Desktop/Research_Paper/EEG Analysis/Sleep_Stage_Detection/sleep-edf-database-expanded-1.0.0/sleep-edf-database-expanded-1.0.0/sleep-telemetry/"
    raw=None
    annot_train=None
    files = os.listdir(EEG_path+subject)
    for file in os.listdir(EEG_path+subject):
        if (file.split('-')[1].split('.')[0] == 'PSG'):
            raw = mne.io.read_raw_edf(EEG_path+subject+'/'+file, stim_channel='Marker', preload=True, verbose=False)
        else:
            annot_train = mne.read_annotations(EEG_path+subject+'/'+file)
            raw.set_annotations(annot_train, emit_warning=False)

    raw.set_channel_types({'EOG horizontal': 'eog', 'EEG Fpz-Cz': 'eeg', 'EEG Pz-Oz': 'eeg', 'EMG submental': 'misc'})
    raw.drop_channels(['Marker'])

    raw.filter(0.3, 40, n_jobs=4)  # Filter the data with a 0.3-40 Hz bandpass
    annot_events = {
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 1,
    'Sleep stage 3': 2,
    'Sleep stage 4': 2,
    'Sleep stage R': 3,
    }


    # keep last 30-min wake events before sleep and first 30-min wake events after
    # sleep and redefine annotations on raw data
    annot_train.crop(annot_train[0]['onset'] - 30 * 60,
                    annot_train[-2]['onset'] + 30 * 60)
    raw_train = raw.copy()
    raw_train.set_annotations(annot_train, emit_warning=False)
    events_train, _ = mne.events_from_annotations(raw_train, event_id = annot_events, chunk_duration=30.)

    # create a new event_id that unifies stages 3 and 4
    event_id = {'Sleep stage W': 0,
                'Sleep stage 1/2': 1,
                'Sleep stage 3/4': 2,
                'Sleep stage R': 3}

    # plot events
    # fig = mne.viz.plot_events(events_train, event_id=event_id,
    #                         sfreq=raw_train.info['sfreq'],
    #                         first_samp=events_train[0, 0])

    # keep the color-code for further plotting
    stage_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    # Create epochs from the event data with 30sec time window
    tmax = 30. - 1. / raw_train.info['sfreq']  # tmax in included

    epochs_train = mne.Epochs(raw=raw_train, events=events_train,
                            event_id=event_id, tmin=0., tmax=tmax, baseline=None, preload=True)
    psds, freqs = mne.time_frequency.psd_welch(epochs_train, fmin=0.5, fmax=40., verbose=False)



    n_epochs = len(epochs_train)
    n_channels = len(epochs_train.ch_names)
    epoch_length = tmax

    svd_entropies = np.zeros((n_epochs, n_channels))
    hfd = np.zeros((n_epochs, n_channels))
    perm_ent = np.zeros((n_epochs, n_channels))
    det_fluc = np.zeros((n_epochs, n_channels))


    for i, epoch in enumerate(epochs_train.get_data()):
        for j, channel_data in enumerate(epoch):
            svd_entropies[i, j] = ant.svd_entropy(channel_data, normalize=True)
            hfd[i, j] = ant.higuchi_fd(channel_data)
            perm_ent[i, j] = ant.perm_entropy(channel_data, order=3, normalize=True)
            det_fluc[i, j] = ant.detrended_fluctuation(channel_data)

    X_svd = svd_entropies
    X_hfd = hfd
    X_perm = perm_ent
    X_det = det_fluc
    Y = epochs_train.events[:, -1]
    X_PSD = psds

    data = epochs_train.get_data()

    mean = np.mean(data, axis=2)
    std_dev = np.std(data, axis=2)
    skewness = skew(data, axis=2)
    kurt = kurtosis(data, axis=2)

    X_time_domain = np.concatenate((mean, std_dev, skewness, kurt), axis=1)

    return X_svd, X_hfd, X_perm, X_det, X_PSD, X_time_domain, Y

In [36]:
X_all_svd, X_all_hfd, X_all_perm, X_all_det, X_all_PSD, X_all_time, Y_all = [], [], [], [], [], [], []
EEG_path = "C:/Users/utkar/Desktop/Research_Paper/EEG Analysis/Sleep_Stage_Detection/sleep-edf-database-expanded-1.0.0/sleep-edf-database-expanded-1.0.0/sleep-telemetry/"

for subject in tqdm(os.listdir(EEG_path)):
    tqdm.write(subject, end='\r')
    X1, X2, X3, X4, X5, X6, Y = process_edf(subject)
    X_all_svd.append(X1)
    X_all_hfd.append(X2)
    X_all_perm.append(X3)
    X_all_det.append(X4)
    X_all_PSD.append(X5)
    X_all_time.append(X6)
    Y_all.append(Y)

  0%|          | 0/43 [00:00<?, ?it/s]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    2.7s finished


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1042 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1042 events and 3000 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.0s finished


...

Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
931 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 931 events and 3000 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s finished


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
943 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 943 events and 3000 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s finished


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
923 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 923 events and 3000 original time points ...
0 bad epochs dropped


In [37]:
X_all_time_arr = np.vstack(X_all_time)
X_all_svd_arr = np.vstack(X_all_svd)
X_all_hfd_arr = np.vstack(X_all_hfd)

X_all_perm_arr = np.vstack(X_all_perm)
X_all_det_arr = np.vstack(X_all_det)
X_all_PSD_arr = np.vstack(X_all_PSD)
X_all_PSD_arr = X_all_PSD_arr.reshape(41490, -1)
Y_all_arr = np.concatenate(Y_all)

In [38]:
print(X_all_time_arr.shape, X_all_svd_arr.shape, X_all_hfd_arr.shape, X_all_perm_arr.shape, X_all_det_arr.shape, X_all_PSD_arr.shape, Y_all_arr.shape)

(41490, 16) (41490, 4) (41490, 4) (41490, 4) (41490, 4) (41490, 202) (41490,)


In [39]:
df_X_time = pd.DataFrame(X_all_time_arr)
df_X_svd = pd.DataFrame(X_all_svd_arr)
df_X_hfd = pd.DataFrame(X_all_hfd_arr)
df_X_perm = pd.DataFrame(X_all_perm_arr)
df_X_det = pd.DataFrame(X_all_det_arr)

df_X_PSD = pd.DataFrame(X_all_PSD_arr)
df_Y = pd.DataFrame(Y_all_arr)


df_X_svd.columns = ['EEG Fpz-Cz_svd', 'EEG Pz-Oz_svd', 'EOG horizontal_svd', 'EMG submental_svd']
df_X_hfd.columns = ['EEG Fpz-Cz_hfd', 'EEG Pz-Oz_hfd', 'EOG horizontal_hfd', 'EMG submental_hfd']
df_X_perm.columns = ['EEG Fpz-Cz_perm', 'EEG Pz-Oz_perm', 'EOG horizontal_perm', 'EMG submental_perm']
df_X_det.columns = ['EEG Fpz-Cz_det', 'EEG Pz-Oz_det', 'EOG horizontal_det', 'EMG submental_det']
df_X_time.columns = ['EEG Fpz-Cz_mean', 'EEG Pz-Oz_mean', 'EOG horizontal_mean', 'EMG submental_mean',\
                        'EEG Fpz-Cz_std', 'EEG Pz-Oz_std', 'EOG horizontal_std', 'EMG submental_std',\
                        'EEG Fpz-Cz_skew', 'EEG Pz-Oz_skew', 'EOG horizontal_skew', 'EMG submental_skew',\
                        'EEG Fpz-Cz_kurt', 'EEG Pz-Oz_kurt', 'EOG horizontal_kurt', 'EMG submental_kurt']
df_Y.columns = ['Sleep Stage']

In [40]:
df_input = pd.concat([df_X_svd, df_X_hfd, df_X_perm, df_X_det, df_X_PSD, df_X_time, df_Y ], axis=1)

In [41]:
df_input

,EEG Fpz-Cz_svd,EEG Pz-Oz_svd,EOG horizontal_svd,EMG submental_svd,EEG Fpz-Cz_hfd,EEG Pz-Oz_hfd,EOG horizontal_hfd,EMG submental_hfd,EEG Fpz-Cz_perm,EEG Pz-Oz_perm,...,EMG submental_std,EEG Fpz-Cz_skew,EEG Pz-Oz_skew,EOG horizontal_skew,EMG submental_skew,EEG Fpz-Cz_kurt,EEG Pz-Oz_kurt,EOG horizontal_kurt,EMG submental_kurt,Sleep Stage
0,0.987039,0.981167,0.793655,0.999226,1.729953,1.892506,1.756279,1.954969,0.946917,0.955711,...,0.000124,-3.075678,1.332415,-0.821078,-6.740439,291.523782,201.569396,13.635766,338.614692,0
1,0.525045,0.530804,0.367215,0.998277,1.527066,1.838841,1.621904,2.022486,0.971058,0.983652,...,0.000020,1.477339,1.045068,0.098621,-0.657815,3.276232,4.301412,-0.663105,7.638272,0
2,0.575597,0.671199,0.439653,0.987337,1.478273,1.847214,1.835012,2.008475,0.966203,0.989973,...,0.000023,2.178262,-0.424238,0.756337,-0.020747,7.854767,4.168572,0.916823,5.337252,0
3,0.526120,0.614145,0.596802,0.999513,1.576452,1.838328,1.883132,2.011644,0.978298,0.987571,...,0.000019,0.972742,1.240378,-0.476201,-0.250570,3.591780,5.138569,-0.413813,7.690777,0
4,0.376618,0.498464,0.423870,0.908455,1.520067,1.809161,1.756443,2.015401,0.985030,0.960600,...,0.000014,1.527443,0.353598,-0.498222,0.246536,1.632868,6.017006,-0.108143,2.962941,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41485,0.460974,0.942977,0.348455,0.388256,1.422695,1.915871,1.389126,1.768020,0.968626,0.993327,...,0.000133,2.613621,0.080913,0.370055,-1.499241,10.960019,0.914838,0.754730,3.175698,0
41486,0.513377,0.888967,0.426211,0.522939,1.453040,1.914228,1.500767,1.903348,0.975148,0.993540,...,0.000127,2.208662,-0.739897,0.030930,-1.540903,6.717767,3.091480,1.020134,5.578316,0
41487,0.517469,0.919033,0.337180,0.633221,1.406252,1.899414,1.299624,1.946126,0.956023,0.993835,...,0.000088,1.669194,0.000556,-0.080864,0.133517,5.362088,0.955720,2.094039,0.173725,0
41488,0.529215,0.796246,0.455389,0.897802,1.514632,1.884125,1.552587,1.969740,0.978405,0.995693,...,0.000090,2.045903,1.460704,1.300098,0.463508,6.598154,5.314809,4.117882,3.020165,0


In [42]:
df_input = df_input.fillna(0)

In [43]:
clf = setup(
    data = df_input,
    target = 'Sleep Stage',
    normalize = True,
    fix_imbalance=True
    )

,Description,Value
0,session_id,8959
1,Target,Sleep Stage
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(41490, 235)"
5,Missing Values,False
6,Numeric Features,234
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
compare_models(fold=15)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.8744,0.9793,0.8547,0.8777,0.8755,0.8234,0.8238,177.7513
lightgbm,Light Gradient Boosting Machine,0.8714,0.9791,0.8561,0.8764,0.8730,0.8198,0.8204,12.3633
rf,Random Forest Classifier,0.8614,0.9743,0.8396,0.8661,0.8630,0.8051,0.8055,15.4633
et,Extra Trees Classifier,0.8600,0.9732,0.8329,0.8612,0.8601,0.8021,0.8023,9.6213
gbc,Gradient Boosting Classifier,0.8417,0.9717,0.8405,0.8553,0.8454,0.7810,0.7833,223.0213
svm,SVM - Linear Kernel,0.7922,0.0000,0.7954,0.8166,0.7979,0.7159,0.7207,5.8793
knn,K Neighbors Classifier,0.7860,0.9295,0.7940,0.8227,0.7940,0.7109,0.7187,18.8880
lda,Linear Discriminant Analysis,0.7688,0.9387,0.7622,0.7957,0.7771,0.6826,0.6863,6.2940
dt,Decision Tree Classifier,0.7642,0.8410,0.7402,0.7770,0.7688,0.6722,0.6736,8.5440
ridge,Ridge Classifier,0.7401,0.0000,0.7493,0.7739,0.7459,0.6492,0.6572,5.0680


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=7075, reg_alpha=0, ...)

In [44]:
xgb_2 = create_model('xgboost', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8926,0.9769,0.9106,0.8959,0.8930,0.8329,0.8342
1,0.8926,0.9775,0.9002,0.8945,0.8929,0.8316,0.8322
2,0.9050,0.9788,0.9093,0.9060,0.9052,0.8503,0.8506
3,0.9086,0.9804,0.9238,0.9114,0.9089,0.8571,0.8580
4,0.8884,0.9750,0.9004,0.8905,0.8887,0.8255,0.8264
5,0.9060,0.9788,0.9104,0.9068,0.9061,0.8518,0.8520
6,0.9050,0.9800,0.9120,0.9064,0.9052,0.8507,0.8512
7,0.8936,0.9766,0.9041,0.8952,0.8938,0.8329,0.8334
8,0.8967,0.9766,0.9136,0.9004,0.8971,0.8393,0.8407


In [45]:
et_2 = create_model('et', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8761,0.9676,0.8951,0.8816,0.8768,0.8073,0.8088
1,0.8673,0.9683,0.8772,0.8719,0.8680,0.7924,0.7934
2,0.8765,0.9668,0.8930,0.8799,0.8769,0.8074,0.8085
3,0.8755,0.9704,0.8975,0.8815,0.8760,0.8067,0.8084
4,0.8802,0.9661,0.8905,0.8826,0.8805,0.8122,0.8129
5,0.8786,0.9715,0.8880,0.8810,0.8790,0.8097,0.8104
6,0.8771,0.9712,0.8889,0.8804,0.8774,0.8072,0.8080
7,0.8683,0.9664,0.8800,0.8734,0.8691,0.7942,0.7953
8,0.8771,0.9692,0.8991,0.8842,0.8782,0.8095,0.8115


In [46]:
lgbm = create_model('lightgbm', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8957,0.9761,0.9124,0.8984,0.8960,0.8374,0.8385
1,0.8890,0.9781,0.8995,0.8919,0.8895,0.8267,0.8277
2,0.9039,0.9789,0.9095,0.9052,0.9042,0.8488,0.8492
3,0.8998,0.9792,0.9227,0.9044,0.9002,0.8447,0.8465
4,0.8827,0.9735,0.9035,0.8867,0.8831,0.8182,0.8199
5,0.9029,0.9777,0.9096,0.9040,0.9031,0.8472,0.8476
6,0.9070,0.9801,0.9217,0.9097,0.9073,0.8549,0.8560
7,0.8982,0.9757,0.9126,0.9010,0.8986,0.8410,0.8419
8,0.8977,0.9755,0.9172,0.9018,0.8981,0.8412,0.8429


In [47]:
rf = create_model('rf', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8745,0.9683,0.8959,0.8807,0.8753,0.8055,0.8074
1,0.8807,0.9714,0.8915,0.8848,0.8814,0.8135,0.8145
2,0.8941,0.9715,0.9071,0.8973,0.8945,0.8342,0.8350
3,0.8822,0.9737,0.9019,0.8880,0.8830,0.8171,0.8188
4,0.8833,0.9680,0.8993,0.8866,0.8837,0.8179,0.8190
5,0.8931,0.9734,0.8984,0.8946,0.8933,0.8315,0.8318
6,0.8915,0.9732,0.9036,0.8944,0.8919,0.8299,0.8307
7,0.8765,0.9681,0.8889,0.8815,0.8775,0.8074,0.8086
8,0.8843,0.9703,0.9041,0.8908,0.8853,0.8206,0.8225


## 3 Stages - W, NREM and REM

In [54]:
def process_edf(subject):
    EEG_path = "C:/Users/utkar/Desktop/Research_Paper/EEG Analysis/Sleep_Stage_Detection/sleep-edf-database-expanded-1.0.0/sleep-edf-database-expanded-1.0.0/sleep-telemetry/"
    raw=None
    annot_train=None
    files = os.listdir(EEG_path+subject)
    for file in os.listdir(EEG_path+subject):
        if (file.split('-')[1].split('.')[0] == 'PSG'):
            raw = mne.io.read_raw_edf(EEG_path+subject+'/'+file, stim_channel='Marker', preload=True, verbose=False)
        else:
            annot_train = mne.read_annotations(EEG_path+subject+'/'+file)
            raw.set_annotations(annot_train, emit_warning=False)

    raw.set_channel_types({'EOG horizontal': 'eog', 'EEG Fpz-Cz': 'eeg', 'EEG Pz-Oz': 'eeg', 'EMG submental': 'misc'})
    raw.drop_channels(['Marker'])

    raw.filter(0.3, 40, n_jobs=4)  # Filter the data with a 0.3-40 Hz bandpass
    annot_events = {
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 1,
    'Sleep stage 3': 1,
    'Sleep stage 4': 1,
    'Sleep stage R': 2,
    }


    # keep last 30-min wake events before sleep and first 30-min wake events after
    # sleep and redefine annotations on raw data
    annot_train.crop(annot_train[0]['onset'] - 30 * 60,
                    annot_train[-2]['onset'] + 30 * 60)
    raw_train = raw.copy()
    raw_train.set_annotations(annot_train, emit_warning=False)
    events_train, _ = mne.events_from_annotations(raw_train, event_id = annot_events, chunk_duration=30.)

    # create a new event_id that unifies stages 3 and 4
    event_id = {'Sleep stage W': 0,
                'Sleep stage 1/2/3/4': 1,
                'Sleep stage R': 2}

    # plot events
    # fig = mne.viz.plot_events(events_train, event_id=event_id,
    #                         sfreq=raw_train.info['sfreq'],
    #                         first_samp=events_train[0, 0])

    # keep the color-code for further plotting
    stage_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    # Create epochs from the event data with 30sec time window
    tmax = 30. - 1. / raw_train.info['sfreq']  # tmax in included

    epochs_train = mne.Epochs(raw=raw_train, events=events_train,
                            event_id=event_id, tmin=0., tmax=tmax, baseline=None, preload=True)
    psds, freqs = mne.time_frequency.psd_welch(epochs_train, fmin=0.5, fmax=40., verbose=False)



    n_epochs = len(epochs_train)
    n_channels = len(epochs_train.ch_names)
    epoch_length = tmax

    svd_entropies = np.zeros((n_epochs, n_channels))
    hfd = np.zeros((n_epochs, n_channels))
    perm_ent = np.zeros((n_epochs, n_channels))
    det_fluc = np.zeros((n_epochs, n_channels))


    for i, epoch in enumerate(epochs_train.get_data()):
        for j, channel_data in enumerate(epoch):
            svd_entropies[i, j] = ant.svd_entropy(channel_data, normalize=True)
            hfd[i, j] = ant.higuchi_fd(channel_data)
            perm_ent[i, j] = ant.perm_entropy(channel_data, order=3, normalize=True)
            det_fluc[i, j] = ant.detrended_fluctuation(channel_data)

    X_svd = svd_entropies
    X_hfd = hfd
    X_perm = perm_ent
    X_det = det_fluc
    Y = epochs_train.events[:, -1]
    X_PSD = psds

    data = epochs_train.get_data()

    mean = np.mean(data, axis=2)
    std_dev = np.std(data, axis=2)
    skewness = skew(data, axis=2)
    kurt = kurtosis(data, axis=2)

    X_time_domain = np.concatenate((mean, std_dev, skewness, kurt), axis=1)

    return X_svd, X_hfd, X_perm, X_det, X_PSD, X_time_domain, Y

In [55]:
X_all_svd, X_all_hfd, X_all_perm, X_all_det, X_all_PSD, X_all_time, Y_all = [], [], [], [], [], [], []
EEG_path = "C:/Users/utkar/Desktop/Research_Paper/EEG Analysis/Sleep_Stage_Detection/sleep-edf-database-expanded-1.0.0/sleep-edf-database-expanded-1.0.0/sleep-telemetry/"

for subject in tqdm(os.listdir(EEG_path)):
    tqdm.write(subject, end='\r')
    X1, X2, X3, X4, X5, X6, Y = process_edf(subject)
    X_all_svd.append(X1)
    X_all_hfd.append(X2)
    X_all_perm.append(X3)
    X_all_det.append(X4)
    X_all_PSD.append(X5)
    X_all_time.append(X6)
    Y_all.append(Y)

  0%|          | 0/43 [00:00<?, ?it/s]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.1s finished


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1042 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1042 events and 3000 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s finished


...

Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
931 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 931 events and 3000 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s finished


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
943 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 943 events and 3000 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1101 samples (11.010 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s finished


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
923 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 923 events and 3000 original time points ...
0 bad epochs dropped


In [56]:
X_all_time_arr = np.vstack(X_all_time)
X_all_svd_arr = np.vstack(X_all_svd)
X_all_hfd_arr = np.vstack(X_all_hfd)

X_all_perm_arr = np.vstack(X_all_perm)
X_all_det_arr = np.vstack(X_all_det)
X_all_PSD_arr = np.vstack(X_all_PSD)
X_all_PSD_arr = X_all_PSD_arr.reshape(41490, -1)
Y_all_arr = np.concatenate(Y_all)

In [57]:
print(X_all_time_arr.shape, X_all_svd_arr.shape, X_all_hfd_arr.shape, X_all_perm_arr.shape, X_all_det_arr.shape, X_all_PSD_arr.shape, Y_all_arr.shape)

(41490, 16) (41490, 4) (41490, 4) (41490, 4) (41490, 4) (41490, 202) (41490,)


In [58]:
df_X_time = pd.DataFrame(X_all_time_arr)
df_X_svd = pd.DataFrame(X_all_svd_arr)
df_X_hfd = pd.DataFrame(X_all_hfd_arr)
df_X_perm = pd.DataFrame(X_all_perm_arr)
df_X_det = pd.DataFrame(X_all_det_arr)

df_X_PSD = pd.DataFrame(X_all_PSD_arr)
df_Y = pd.DataFrame(Y_all_arr)


df_X_svd.columns = ['EEG Fpz-Cz_svd', 'EEG Pz-Oz_svd', 'EOG horizontal_svd', 'EMG submental_svd']
df_X_hfd.columns = ['EEG Fpz-Cz_hfd', 'EEG Pz-Oz_hfd', 'EOG horizontal_hfd', 'EMG submental_hfd']
df_X_perm.columns = ['EEG Fpz-Cz_perm', 'EEG Pz-Oz_perm', 'EOG horizontal_perm', 'EMG submental_perm']
df_X_det.columns = ['EEG Fpz-Cz_det', 'EEG Pz-Oz_det', 'EOG horizontal_det', 'EMG submental_det']
df_X_time.columns = ['EEG Fpz-Cz_mean', 'EEG Pz-Oz_mean', 'EOG horizontal_mean', 'EMG submental_mean',\
                        'EEG Fpz-Cz_std', 'EEG Pz-Oz_std', 'EOG horizontal_std', 'EMG submental_std',\
                        'EEG Fpz-Cz_skew', 'EEG Pz-Oz_skew', 'EOG horizontal_skew', 'EMG submental_skew',\
                        'EEG Fpz-Cz_kurt', 'EEG Pz-Oz_kurt', 'EOG horizontal_kurt', 'EMG submental_kurt']
df_Y.columns = ['Sleep Stage']

In [59]:
df_input = pd.concat([df_X_svd, df_X_hfd, df_X_perm, df_X_det, df_X_PSD, df_X_time, df_Y ], axis=1)

In [60]:
df_input

,EEG Fpz-Cz_svd,EEG Pz-Oz_svd,EOG horizontal_svd,EMG submental_svd,EEG Fpz-Cz_hfd,EEG Pz-Oz_hfd,EOG horizontal_hfd,EMG submental_hfd,EEG Fpz-Cz_perm,EEG Pz-Oz_perm,...,EMG submental_std,EEG Fpz-Cz_skew,EEG Pz-Oz_skew,EOG horizontal_skew,EMG submental_skew,EEG Fpz-Cz_kurt,EEG Pz-Oz_kurt,EOG horizontal_kurt,EMG submental_kurt,Sleep Stage
0,0.987039,0.981167,0.793655,0.999226,1.729953,1.892506,1.756279,1.954969,0.946917,0.955711,...,0.000124,-3.075678,1.332415,-0.821078,-6.740439,291.523782,201.569396,13.635766,338.614692,0
1,0.525045,0.530804,0.367215,0.998277,1.527066,1.838841,1.621904,2.022486,0.971058,0.983652,...,0.000020,1.477339,1.045068,0.098621,-0.657815,3.276232,4.301412,-0.663105,7.638272,0
2,0.575597,0.671199,0.439653,0.987337,1.478273,1.847214,1.835012,2.008475,0.966203,0.989973,...,0.000023,2.178262,-0.424238,0.756337,-0.020747,7.854767,4.168572,0.916823,5.337252,0
3,0.526120,0.614145,0.596802,0.999513,1.576452,1.838328,1.883132,2.011644,0.978298,0.987571,...,0.000019,0.972742,1.240378,-0.476201,-0.250570,3.591780,5.138569,-0.413813,7.690777,0
4,0.376618,0.498464,0.423870,0.908455,1.520067,1.809161,1.756443,2.015401,0.985030,0.960600,...,0.000014,1.527443,0.353598,-0.498222,0.246536,1.632868,6.017006,-0.108143,2.962941,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41485,0.460974,0.942977,0.348455,0.388256,1.422695,1.915871,1.389126,1.768020,0.968626,0.993327,...,0.000133,2.613621,0.080913,0.370055,-1.499241,10.960019,0.914838,0.754730,3.175698,0
41486,0.513377,0.888967,0.426211,0.522939,1.453040,1.914228,1.500767,1.903348,0.975148,0.993540,...,0.000127,2.208662,-0.739897,0.030930,-1.540903,6.717767,3.091480,1.020134,5.578316,0
41487,0.517469,0.919033,0.337180,0.633221,1.406252,1.899414,1.299624,1.946126,0.956023,0.993835,...,0.000088,1.669194,0.000556,-0.080864,0.133517,5.362088,0.955720,2.094039,0.173725,0
41488,0.529215,0.796246,0.455389,0.897802,1.514632,1.884125,1.552587,1.969740,0.978405,0.995693,...,0.000090,2.045903,1.460704,1.300098,0.463508,6.598154,5.314809,4.117882,3.020165,0


In [61]:
df_input = df_input.fillna(0)

In [62]:
clf = setup(
    data = df_input,
    target = 'Sleep Stage',
    normalize = True,
    fix_imbalance=True
    )

,Description,Value
0,session_id,1778
1,Target,Sleep Stage
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(41490, 235)"
5,Missing Values,False
6,Numeric Features,234
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
compare_models(fold=15)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.8744,0.9793,0.8547,0.8777,0.8755,0.8234,0.8238,177.7513
lightgbm,Light Gradient Boosting Machine,0.8714,0.9791,0.8561,0.8764,0.8730,0.8198,0.8204,12.3633
rf,Random Forest Classifier,0.8614,0.9743,0.8396,0.8661,0.8630,0.8051,0.8055,15.4633
et,Extra Trees Classifier,0.8600,0.9732,0.8329,0.8612,0.8601,0.8021,0.8023,9.6213
gbc,Gradient Boosting Classifier,0.8417,0.9717,0.8405,0.8553,0.8454,0.7810,0.7833,223.0213
svm,SVM - Linear Kernel,0.7922,0.0000,0.7954,0.8166,0.7979,0.7159,0.7207,5.8793
knn,K Neighbors Classifier,0.7860,0.9295,0.7940,0.8227,0.7940,0.7109,0.7187,18.8880
lda,Linear Discriminant Analysis,0.7688,0.9387,0.7622,0.7957,0.7771,0.6826,0.6863,6.2940
dt,Decision Tree Classifier,0.7642,0.8410,0.7402,0.7770,0.7688,0.6722,0.6736,8.5440
ridge,Ridge Classifier,0.7401,0.0000,0.7493,0.7739,0.7459,0.6492,0.6572,5.0680


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=7075, reg_alpha=0, ...)

In [63]:
xgb_3 = create_model('xgboost', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9334,0.9838,0.9290,0.9368,0.9343,0.8586,0.8600
1,0.9303,0.9854,0.9288,0.9338,0.9312,0.8521,0.8536
2,0.9468,0.9893,0.9439,0.9492,0.9474,0.8861,0.8870
3,0.9318,0.9869,0.9324,0.9359,0.9328,0.8559,0.8578
4,0.9360,0.9875,0.9404,0.9404,0.9370,0.8646,0.8663
5,0.9329,0.9855,0.9241,0.9348,0.9335,0.8559,0.8565
6,0.9458,0.9899,0.9351,0.9470,0.9461,0.8826,0.8828
7,0.9339,0.9867,0.9314,0.9363,0.9345,0.8585,0.8593
8,0.9421,0.9864,0.9365,0.9443,0.9428,0.8759,0.8765


In [64]:
et_3 = create_model('et', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9102,0.9759,0.9073,0.9161,0.9117,0.8107,0.8126
1,0.9122,0.9770,0.9125,0.9167,0.9133,0.8141,0.8156
2,0.9205,0.9804,0.9231,0.9259,0.9217,0.8314,0.8331
3,0.9168,0.9796,0.9153,0.9211,0.9180,0.8239,0.8254
4,0.9148,0.9780,0.9174,0.9224,0.9165,0.8204,0.8227
5,0.9112,0.9752,0.9030,0.9153,0.9119,0.8095,0.8106
6,0.9215,0.9802,0.9239,0.9268,0.9227,0.8338,0.8356
7,0.9122,0.9776,0.9173,0.9187,0.9135,0.8147,0.8167
8,0.9148,0.9788,0.9146,0.9213,0.9162,0.8194,0.8212


In [65]:
rf_3 = create_model('rf', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9241,0.9780,0.9257,0.9298,0.9256,0.8401,0.8422
1,0.9205,0.9787,0.9191,0.9245,0.9215,0.8312,0.8325
2,0.9267,0.9827,0.9241,0.9311,0.9277,0.8436,0.8448
3,0.9251,0.9824,0.9234,0.9290,0.9261,0.8411,0.8425
4,0.9246,0.9814,0.9302,0.9322,0.9264,0.8415,0.8439
5,0.9189,0.9782,0.9102,0.9221,0.9195,0.8255,0.8262
6,0.9339,0.9835,0.9350,0.9385,0.9350,0.8595,0.8609
7,0.9256,0.9801,0.9294,0.9313,0.9269,0.8426,0.8444
8,0.9199,0.9809,0.9187,0.9253,0.9213,0.8303,0.8319


In [66]:
lgbm_3 = create_model('lightgbm', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9293,0.9834,0.9258,0.9331,0.9303,0.8502,0.8518
1,0.9277,0.9840,0.9335,0.9321,0.9288,0.8476,0.8496
2,0.9437,0.9888,0.9446,0.9466,0.9444,0.8801,0.8813
3,0.9334,0.9861,0.9373,0.9379,0.9344,0.8596,0.8618
4,0.9308,0.9863,0.9388,0.9367,0.9322,0.8547,0.8570
5,0.9241,0.9842,0.9169,0.9274,0.9250,0.8384,0.8395
6,0.9447,0.9885,0.9394,0.9463,0.9452,0.8810,0.8815
7,0.9282,0.9859,0.9327,0.9325,0.9292,0.8482,0.8499
8,0.9334,0.9865,0.9281,0.9360,0.9341,0.8577,0.8586


## 2 Stages - REM and NREM

In [ ]:
df_input = pd.concat([df_X_svd, df_X_hfd, df_X_perm, df_X_det, df_X_PSD, df_X_time, df_Y ], axis=1)

In [67]:
df_input_2 = df_input[df_input['Sleep Stage'] != 0]
df_input_2['Sleep Stage'].value_counts()

1    29193
2     8149
Name: Sleep Stage, dtype: int64

In [72]:
df_input_2['Sleep Stage'] = df_input_2['Sleep Stage'].apply(lambda x: 0 if x == 1 else 1)

In [74]:
df_input_2 = df_input_2.fillna(0)
df_input_2['Sleep Stage'].value_counts()

0    29193
1     8149
Name: Sleep Stage, dtype: int64

In [75]:
clf = setup(
    data = df_input_2,
    target = 'Sleep Stage',
    normalize = True,
    fix_imbalance=True
    )

,Description,Value
0,session_id,2424
1,Target,Sleep Stage
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(37342, 235)"
5,Missing Values,False
6,Numeric Features,234
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
compare_models(fold=15)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.8744,0.9793,0.8547,0.8777,0.8755,0.8234,0.8238,177.7513
lightgbm,Light Gradient Boosting Machine,0.8714,0.9791,0.8561,0.8764,0.8730,0.8198,0.8204,12.3633
rf,Random Forest Classifier,0.8614,0.9743,0.8396,0.8661,0.8630,0.8051,0.8055,15.4633
et,Extra Trees Classifier,0.8600,0.9732,0.8329,0.8612,0.8601,0.8021,0.8023,9.6213
gbc,Gradient Boosting Classifier,0.8417,0.9717,0.8405,0.8553,0.8454,0.7810,0.7833,223.0213
svm,SVM - Linear Kernel,0.7922,0.0000,0.7954,0.8166,0.7979,0.7159,0.7207,5.8793
knn,K Neighbors Classifier,0.7860,0.9295,0.7940,0.8227,0.7940,0.7109,0.7187,18.8880
lda,Linear Discriminant Analysis,0.7688,0.9387,0.7622,0.7957,0.7771,0.6826,0.6863,6.2940
dt,Decision Tree Classifier,0.7642,0.8410,0.7402,0.7770,0.7688,0.6722,0.6736,8.5440
ridge,Ridge Classifier,0.7401,0.0000,0.7493,0.7739,0.7459,0.6492,0.6572,5.0680


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=7075, reg_alpha=0, ...)

In [76]:
xgb_4 = create_model('xgboost', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9598,0.9923,0.9403,0.8851,0.9118,0.8859,0.8865
1,0.9558,0.9889,0.9247,0.8812,0.9024,0.8739,0.8743
2,0.9530,0.9858,0.8831,0.9019,0.8924,0.8623,0.8624
3,0.9604,0.9915,0.9351,0.8911,0.9125,0.8870,0.8874
4,0.9598,0.9887,0.9195,0.9008,0.9100,0.8842,0.8843
5,0.9558,0.9816,0.9143,0.8889,0.9014,0.8729,0.8731
6,0.9530,0.9867,0.9273,0.8686,0.8970,0.8665,0.8673
7,0.9547,0.9882,0.9091,0.8883,0.8986,0.8694,0.8695
8,0.9593,0.9912,0.9273,0.8925,0.9096,0.8833,0.8835


In [77]:
et_4 = create_model('et', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9530,0.9858,0.8987,0.8895,0.8941,0.8638,0.8638
1,0.9472,0.9825,0.8753,0.8845,0.8799,0.8461,0.8461
2,0.9421,0.9795,0.8416,0.8901,0.8652,0.8283,0.8288
3,0.9472,0.9878,0.8779,0.8825,0.8802,0.8464,0.8464
4,0.9512,0.9817,0.8494,0.9237,0.8850,0.8541,0.8553
5,0.9392,0.9784,0.8442,0.8760,0.8598,0.8210,0.8212
6,0.9432,0.9814,0.8701,0.8724,0.8713,0.8348,0.8348
7,0.9421,0.9821,0.8571,0.8777,0.8673,0.8302,0.8303
8,0.9495,0.9852,0.8779,0.8918,0.8848,0.8525,0.8525


In [78]:
lgbm_4 = create_model('lightgbm', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9535,0.9912,0.9299,0.8689,0.8984,0.8683,0.8691
1,0.9478,0.9887,0.9143,0.8585,0.8855,0.8518,0.8525
2,0.9530,0.9853,0.8883,0.8976,0.8930,0.8628,0.8628
3,0.9604,0.9912,0.9532,0.8780,0.9141,0.8884,0.8897
4,0.9558,0.9893,0.9143,0.8889,0.9014,0.8729,0.8731
5,0.9552,0.9837,0.9273,0.8771,0.9015,0.8726,0.8732
6,0.9501,0.9860,0.9247,0.8599,0.8911,0.8588,0.8597
7,0.9587,0.9876,0.9403,0.8808,0.9095,0.8828,0.8836
8,0.9575,0.9914,0.9351,0.8802,0.9068,0.8793,0.8800


In [79]:
rf_4 = create_model('rf', fold=15)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9610,0.9895,0.9195,0.9054,0.9124,0.8873,0.8873
1,0.9507,0.9855,0.8987,0.8804,0.8895,0.8577,0.8578
2,0.9478,0.9828,0.8571,0.9016,0.8788,0.8456,0.8460
3,0.9558,0.9894,0.9091,0.8929,0.9009,0.8725,0.8725
4,0.9524,0.9866,0.8753,0.9059,0.8904,0.8600,0.8602
5,0.9489,0.9836,0.8649,0.9000,0.8821,0.8495,0.8498
6,0.9495,0.9850,0.8831,0.8877,0.8854,0.8530,0.8530
7,0.9558,0.9862,0.8935,0.9053,0.8993,0.8710,0.8711
8,0.9552,0.9893,0.9039,0.8946,0.8992,0.8705,0.8705
